In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import urllib.request

np.random.seed(20)

use_warning = False

def gauss(x, mu, sigma):
    return 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2/sigma/sigma/2)

def gauss_filter_freq(freq, sigma):
    return np.exp(-0.5 * (freq / sigma)**2)

def ensure_response_file(npz_path="response_44_v2a_full.npz"):
    npz_path = Path(npz_path)
    if npz_path.exists():
        return npz_path

    url = "https://raw.githubusercontent.com/DUNE/larnd-sim/develop/larndsim/bin/response_44_v2a_full.npz"
    print(f"Downloading {url} -> {npz_path} ...")
    urllib.request.urlretrieve(url, npz_path.as_posix())
    return npz_path

def deconv(wf, k):
    deconvq = np.fft.ifft(np.fft.fft(wf, n=len(wf)) / np.fft.fft(k, n=len(wf))).real
    return deconvq[:len(wf)+1-len(k)]

def gauss_filter(wf, width, dt):
    '''width
    '''
    wf_fft = np.fft.fft(wf, n=len(wf))
    kt = np.arange(len(wf), dtype=np.float64)
    freq = np.fft.fftfreq(len(wf), d=dt)
    gfilter = gauss_filter_freq(freq, width)

    return np.fft.ifft(wf_fft * gfilter).real

def deconv_filter(wf, kernel, width, dt):
    '''width
    T is the sampling rate, which is not used here.
    '''
    wf_fft = np.fft.fft(wf, n=len(wf))
    kernel_fft = np.fft.fft(kernel, n=len(wf))
    kt = np.arange(len(wf), dtype=np.float64)
    freq = np.fft.fftfreq(len(wf), d=dt)
    gfilter = gauss_filter_freq(freq, width)

    return np.fft.ifft(wf_fft / kernel_fft * gfilter).real[:len(wf)-len(kernel)+2]

def field_response(npz_path="response_44_v2a_full.npz"):
    path = ensure_response_file(npz_path)
    return np.load(path)

def trigger(wf, thres, noise=0):
    return np.argmax((np.cumsum(wf)+noise * np.random.normal(0, 1, wf.shape))>thres)

def integrate_k(wf, k):
    if wf.shape[0] % k:
        # raise ValueError('wf.shape[0] % k != 0')
        wf = np.pad(wf, (0, k - wf.shape[0] % k))
        if use_warning:
            print('warning: length of wf is now', len(wf))
    return wf.reshape(-1, k).sum(axis=-1)

def add_noise(wf, amp):
    noise = amp*np.random.normal(0, 1, (len(wf) +1 ))
    noise = np.diff(noise)
    return wf + noise, np.sum(noise)

# integrate every k ticks
def trigger_integrate_k(wf, k, start_idx):
    if wf[start_idx:].shape[0] % k:
        # raise ValueError('wf.shape[0] % k != 0')
        wfrest = np.pad(wf[start_idx:], (0, k - wf[start_idx:].shape[0] % k))
        if use_warning:
            print('warning: length of wf is now', len(wf))
    else:
        wfrest = wf[start_idx:]
    wfrest = wfrest.reshape(-1, k).sum(axis=-1)
    wf_trunc = np.zeros(len(wfrest)+1)
    wf_trunc[0] += np.sum(wf[:start_idx])
    wf_trunc[1:] += wfrest
    return wf_trunc, start_idx

def lost_waveform(wf_k_squeeze, kticks, fr_fg, thres, start_idx_fg):
    ''' assume fr is full length without downsampling
    wf_k_squeeze is 0 to trigger, trigger to end, and burst
    '''
    qmax = np.max(np.cumsum(wf_k_squeeze))
    # print('qmax', qmax)
    idx = np.argmin(np.abs(thres/qmax - np.cumsum(fr_fg)/np.sum(fr_fg)))
    # print(idx, np.cumsum(fr_fg)[idx], thres/qmax)
    lost = np.zeros(start_idx_fg) # at threshold
    if idx < start_idx_fg:
        # print('ok, good')
        lost[-idx:] = fr_fg[:idx] * thres / np.sum(fr_fg[:idx])
        # print(np.sum(lost))
    else:
        print('not good')
        lost = fr_fg[idx-start_idx_fg:idx] * thres / fr_fg[idx]
    ncycles = len(lost) // kticks
    # print(ncycles, len(wf_trunc))
    wf_full = np.zeros(ncycles + len(wf_k_squeeze)-1)
    wf_full[:ncycles] = lost[-ncycles * kticks:].reshape(ncycles, kticks).sum(axis=-1)
    # wf_full[ncycles] = wf_k_squeeze[0]
    # print(wf_full[ncycles], wf_trunc[0], thres)
    wf_full[ncycles:] = wf_k_squeeze[1:]
    return wf_full

def scale_q_test(qscale, shift=0, kticks=30, thres=5, noise=1, use_filter: None|dict = None, show: bool =True):
    q = gauss(np.arange(-10, 10, 0.1), 0, 0.5)
    q /= np.sum(q)
    qunit = gauss(np.arange(-10, 10, 0.1), 0, 0.5)
    qunit /= np.sum(qunit)

    q *= qscale
    q = np.roll(q, shift)
    # if show:
    #     plt.figure()
    #     plt.title('Charge distribution')
    #     plt.plot(q)
    fr0 = field_response()['response'][0,0]
    fr0 = fr0.reshape(-1, 2).sum(axis=-1)[-1801:]
    fr0 *= 0.05

    wf = np.convolve(q, fr0, mode='full')

    deconvq = deconv(wf, fr0)

    wf_k_squeeze, start_idx_fg = trigger_integrate_k(wf, kticks, trigger(wf, thres, noise=noise)+1)
    # print(wf_k_squeeze, start_idx_fg, len(wf) - start_idx_fg,  np.sum(wf[start_idx_fg+1:start_idx_fg+kticks+1]), np.sum(wf[:start_idx_fg+1]))
    wf_k_squeeze[1:], total_noise = add_noise(wf_k_squeeze[1:], noise)
    fr0_k = integrate_k(fr0, kticks)

    # print('----------------------: ', np.sum(np.convolve(qunit, fr0)))

    qfr0_k = integrate_k(np.convolve(qunit, fr0), kticks)
    wf_full_k = lost_waveform(wf_k_squeeze, kticks, fr0, thres, start_idx_fg)
    if use_filter:
        qdeconv2 = deconv_filter(wf_full_k, fr0_k, width=use_filter['width'], dt=kticks)
        qsmear = gauss_filter(q, width=use_filter['width'], dt=1)
    else:
        qdeconv2 = deconv(wf_full_k, fr0_k)

    if show:
        plt.figure()
        plt.plot(np.arange(len(wf_full_k))*kticks, wf_full_k, label='guess')
        plt.plot(wf*kticks, label='wf true')
        plt.legend()
        plt.show()

    if show:
        plt.figure()
        plt.plot(np.arange(len(qdeconv2))*kticks, qdeconv2/kticks, label='average')
        # plt.plot(deconvq, label='qhat')
        plt.plot(q, label='true')
        if use_filter:
            plt.plot(qsmear, label=f'smeared true; sigma={1/2/np.pi/use_filter["width"]:.1f} ticks')
        plt.legend()
        plt.title(f'Injected charge {qscale:.1f}ke-')
        plt.show()

    return np.sum(qdeconv2) - np.sum(q), np.argmax(qdeconv2)*kticks - np.argmax(q), total_noise

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def fit_line_band(x, y, k=2.0, use_vertical_residual=True):
    """
    """
    x = np.asarray(x).ravel()
    y = np.asarray(y).ravel()
    if x.size != y.size:
        raise ValueError("len(x) must be equal to len(y)")

    # 1) y = a*x + b
    a, b = np.polyfit(x, y, deg=1)

    # 2) residue
    y_hat = a * x + b
    r = y - y_hat

    if use_vertical_residual:
        # vertical residue
        sigma = np.sqrt(np.mean(r**2))
    else:
        # from point to line：|ax - y + b| / sqrt(a^2 + 1)
        d = np.abs(a * x - y + b) / np.sqrt(a**2 + 1.0)
        sigma = np.sqrt(np.mean(d**2))

    band_half = k * sigma
    return a, b, sigma, band_half

def plot_line_band(x, y, a, b, band_half, title=None, labels={}):
    x = np.asarray(x).ravel()
    y = np.asarray(y).ravel()

    # sorted
    xs = np.linspace(x.min(), x.max(), 300)
    ys = a * xs + b

    plt.figure(figsize=(7, 5))
    plt.scatter(x, y, s=25, alpha=0.8, label=labels.get('data', 'data'))
    plt.plot(xs, ys, linewidth=2, label=f"fit: y={a:.3f}x+{b:.3f}")
    plt.fill_between(xs, ys - band_half, ys + band_half, alpha=0.2,
                     label=f"band: +/-{band_half:.3f}")

    plt.xlabel(labels.get('xaxis', 'x'))
    plt.ylabel(labels.get('yaxis', 'y'))
    if title:
        plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
scale_q_test(24, show=True, kticks=30, use_filter={'width': 0.01})

In [ ]:
delta_qs2 = []
delta_ts2 = []
total_noises2 = []
width_in_true = 0.01
kticks=30
for qscale in np.arange(6, 100, 0.5):
    delta_q, delta_t, total_noise = scale_q_test(qscale, show=False, kticks=kticks, use_filter={'width': width_in_true})
    delta_qs2.append(delta_q)
    delta_ts2.append(delta_t)
    total_noises2.append(total_noise)

In [ ]:
a2, b2, sigma, band_half = fit_line_band(np.arange(6, 100, 0.5), delta_qs2, k=1.0, use_vertical_residual=True)
plot_line_band(np.arange(6, 100, 0.5), delta_qs2, a2, b2, band_half=band_half,
               title=f"Gaussian filter, width = {width_in_true}", labels={'data':'with filter', 'yaxis': 'dQ[ke]', 'xaxis': "injected Q[ke]"}
               )
plt.figure()
plt.scatter(total_noises2, delta_qs2)
plt.xlabel('added noise')
plt.ylabel('delta Q')
# plt.ylim(-15, 15)

In [ ]:
delta_qs = []
delta_ts = []
total_noises = []
for qscale in np.arange(6, 100, 0.5):
    delta_q, delta_t, delta_noise = scale_q_test(qscale, kticks=kticks, noise=1, show=False)
    delta_qs.append(delta_q)
    delta_ts.append(delta_t)
    total_noises.append(delta_noise)

In [ ]:
a, b, sigma, band_half = fit_line_band(np.arange(6, 100, 0.5), delta_qs, k=1.0, use_vertical_residual=True)
plot_line_band(np.arange(6, 100, 0.5), delta_qs, a, b, band_half=band_half, labels={'data':'without filter', 'yaxis': 'dQ[ke]', 'xaxis': "injected Q[ke]" })

plt.figure()
plt.scatter(total_noises, delta_qs)
plt.xlabel('added noise')
plt.ylabel('delta Q')

In [ ]:
plt.figure()
plt.scatter(total_noises, delta_qs, label='without filter')
plt.scatter(total_noises2, delta_qs2, label='with filter')
plt.legend()
plt.xlabel('added noise')
plt.ylabel('delta Q')

In [ ]:
import numpy as np

In [ ]:
fr0 = field_response()['response'][0,0]
fr0 = fr0.reshape(-1, 2).sum(axis=-1)[-1801:]
fr0 *= 0.05

In [ ]:
fr0_k = integrate_k(fr0, k=30)

In [ ]:
m = 6400//30
fr0_k_freq = np.fft.fftfreq(n=m, d=0.05*30)
qgaus = 2.4*gauss(np.arange(-10, 10, 0.1), 0, 0.5)
qgaus = qgaus/np.sum(qgaus)
qgaus_k = integrate_k(qgaus, k=30)
wf_true = np.convolve(qgaus, fr0, mode='full')
wf_true_k = integrate_k(wf_true, k=30)
plt.figure()
plt.plot(fr0_k_freq, np.abs(np.fft.fft(fr0_k, n=m)), 'o', label='fr0_k')
plt.plot(np.fft.fftfreq(n=m, d=0.05), np.abs(np.fft.fft(qgaus, n=m)), 'o', label='qgaus')
plt.plot(fr0_k_freq, np.abs(np.fft.fft(qgaus_k, n=m)), 'o', label='qgaus_k')
plt.plot(fr0_k_freq, np.abs(np.fft.fft(wf_true_k, n=m)), 'o', label='wf_true_k')
plt.hlines([1,], -0.3, 0.3, linestyles='--')
plt.plot(fr0_k_freq, gauss_filter_freq(fr0_k_freq, sigma=0.01), 'o', label='gaus sigma=0.01')
plt.xlim(-1, 1)
plt.xlabel('Frequency [MHz]')
plt.legend()
# plt.figure()
# plt.plot(np.fft.fftfreq(n=6000//30, d=0.05*30), 'o')
plt.figure()
plt.plot(gauss_filter_freq(fr0_k_freq, sigma=0.01))

In [ ]:
plt.plot(np.fft.fftfreq(n=3000, d=0.05), np.abs(np.fft.fft(fr0, n=3000)), 'o')